# Matrix Decomposition

This notebook covers common matrix decomposition methods used in machine learning:
- QR Decomposition
- Eigen Decomposition
- Singular Value Decomposition (SVD)

In [2]:
import numpy as np

In [4]:
def qr_decomposition(vectors):
    """
    Perform QR decomposition via Gram-Schmidt on a set of vectors

    Args:
        vectors: A numpy matrix n by m, where each column is a vector of length n

    Returns:
        Q, R where Q = orthonormal basis and R = upper triangular matrix of scalar projections
        and A = Q * R
    """
    n, m = vectors.shape
    A = vectors.copy().astype(float)
    Q = np.zeros((n, m)).astype(float)
    R = np.zeros((m, m)).astype(float)

    # We can normalize beforehand to simplify projecting each vector onto every other vector
    # But that may lead to floating point errors, so let's prefer the standard method
    for j1 in range(m):
        vec_j1 = A[:, j1].copy()

        # Subtract projections of vec_j1 on previous vectors
        ortho_vec_j1 = A[:, j1].copy()
        for j2 in range(j1):
            vec_j2 = Q[:, j2].copy()

            # Normalize vec_j2
            mag_j2 = np.sqrt(np.sum(vec_j2 * vec_j2))
            norm_vec_j2 = vec_j2 / mag_j2

            # Get the scalar projection of vec_j1 onto vec_j2:
            # The scalar projection is typically vec_j1 @ vec_j2 / ||vec_j2||
            # But we have the normalized vector of vec_j2
            # And vec_j1 @ norm_vec_j2 = ||vec_j1|| * ||norm_vec_j2|| * cos(theta) = ||vec_j1|| * cos(theta)
            # Because vec_j1 is the hypotenus and cos(theta) = adj / hyp
            # Thus, ||vec_j1|| * cos(theta) = length of adj = length of vec_j1 projected onto norm_vec_j2
            # "A unit vector’s components are the cosines of its angles with the coordinate axes."
            scalar_proj_j1 = np.dot(vec_j1, norm_vec_j2)
            
            # Store the scalar projection of the original vector onto norm_vec_j2
            R[j2][j1] = scalar_proj_j1

            # Project vec_j1 onto vec_j2
            proj_j1_onto_j2 = scalar_proj_j1 * norm_vec_j2

            # Subtract the projection from ortho_vec_j1
            ortho_vec_j1 -= proj_j1_onto_j2

        # Normalize ortho_vec_j1
        mag_ortho_j1 = np.sqrt(np.sum(ortho_vec_j1 * ortho_vec_j1))
        orthonorm_vec_j1 = ortho_vec_j1 / mag_ortho_j1
        R[j1][j1] = mag_ortho_j1

        # Add orthonormal basis vector to Q
        Q[:, j1] = orthonorm_vec_j1

    return Q, R


sample_vectors = np.array([[1, 2], 
                           [0, 2]])
Q, R = qr_decomposition(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)

print('\n===\n')

sample_vectors = np.array([[1, 1, 0], 
                           [0, 1, 1], 
                           [0, 0, 1]])
Q, R = qr_decomposition(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)

print('\n===\n')

sample_vectors = np.array([[1, 1, 0], 
                           [1, 0, 1], 
                           [0, 1, 1]])
Q, R = qr_decomposition(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)

Q: 
 [[1. 0.]
 [0. 1.]]
R: 
 [[1. 2.]
 [0. 2.]]

===

Q: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
R: 
 [[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]]

===

Q: 
 [[ 0.70710678  0.40824829 -0.57735027]
 [ 0.70710678 -0.40824829  0.57735027]
 [ 0.          0.81649658  0.57735027]]
R: 
 [[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]]


In [9]:
def eigendecomposition(A):
    """
    Performs eigendecomposition on a square matrix A

    Args:
        A: A 2D np.array (n, n), the square matrix to decompose

    Returns:
        V,  where V = eigenvectors and R = diagonalized matrix of eigenvalues
        and A = V * lambdas * V^-1
    """
    n = A.shape[0]
    V = np.zeros((n, n)).astype(float)
    lambdas = np.zeros((n, n)).astype(float)
    V_inverse = np.zeros((n, n)).astype(float)

    # Getting eigenvalues by hand means getting the roots of the characteristic polynomial
    # for A - lambda * I. Eigenvectors of a linear transformation are vectors that are on the same span
    # after applying the transformation and the eigenvalues are the multiplier that they scale by. Or
    # mathematically, vectors such that A * v = lambda * v. For each eigenvalue, you want to find the 
    # null space of A - lambda * I that gives you the eigenvectors.
    eigenvals, eigenvecs = np.linalg.eig(A)

    for i in range(n):
        V[:,i] = eigenvecs[:,i]
        lambdas[i][i] = eigenvals[i]

    V_inverse = np.linalg.inv(V)

    return V, lambdas, V_inverse

sample_A = np.array([[1, 0, 0], 
                           [0, 1, 0], 
                           [0, 0, 1]])
V, lambdas, V_inverse = eigendecomposition(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', V @ lambdas @ V_inverse)

print('\n===\n')

sample_A = np.array([[2, 0, 0], 
                           [0, 3, 0], 
                           [0, 0, 4]])
V, lambdas, V_inverse = eigendecomposition(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', V @ lambdas @ V_inverse)

print('\n===\n')

sample_A = np.array([[2, 1, 0], 
                           [1, 2, 1], 
                           [0, 1, 2]])
V, lambdas, V_inverse = eigendecomposition(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', np.round(V @ lambdas @ V_inverse, decimals=10))

V: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
lambdas: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V * lambdas * V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

===

V: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
lambdas: 
 [[2. 0. 0.]
 [0. 3. 0.]
 [0. 0. 4.]]
V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V * lambdas * V_inverse: 
 [[2. 0. 0.]
 [0. 3. 0.]
 [0. 0. 4.]]

===

V: 
 [[-5.00000000e-01  7.07106781e-01  5.00000000e-01]
 [-7.07106781e-01  4.05405432e-16 -7.07106781e-01]
 [-5.00000000e-01 -7.07106781e-01  5.00000000e-01]]
lambdas: 
 [[3.41421356 0.         0.        ]
 [0.         2.         0.        ]
 [0.         0.         0.58578644]]
V_inverse: 
 [[-5.00000000e-01 -7.07106781e-01 -5.00000000e-01]
 [ 7.07106781e-01  3.14018492e-16 -7.07106781e-01]
 [ 5.00000000e-01 -7.07106781e-01  5.00000000e-01]]
V * lambdas * V_inverse: 
 [[ 2.  1. -0.]
 [ 1.  2.  1.]
 [-0.  1.  2.]]
